# Deploy the Application Package

Deploying an Application Package using the OGC API Processes API uses the API resource highlighted in the table below:


| **Resource**                   | **Path**                                  | **Purpose**                                                                     | **Part**   |
|--------------------------------|-------------------------------------------|---------------------------------------------------------------------------------|------------|
| Landing page                   | `/`                                       | Top-level resource serving as an entry point.                                   | Part 1     |
| Conformance declaration        | `/conformance`                            | Information about the functionality supported by the server.                    | Part 1     |
| API Definition                 | `/api`                                    | Metadata about the API itself.                                                  | Part 1     |
| Process list                   | `/processes`                              | Lists available processes with identifiers and links to descriptions.           | Part 1     |
| Process description            | `/processes/{processID}`                  | Retrieves detailed information about a specific process.                        | Part 1     |
| Process execution              | `/processes/{processID}/execution` (POST) | Executes a process, creating a job.                                             | Part 1     |
| **Deploy Process**             | **`/processes` (POST)**                   | **Deploys a new process on the server.**                                        | **Part 2** |
| Replace Process                | `/processes/{processID}` (PUT)            | Replaces an existing process with a new version.                                | Part 2     |
| Undeploy Process               | `/processes/{processID}` (DELETE)         | Removes an existing process from the server.                                    | Part 2     |
| EO Application Package         | `/processes/{processID}/package`          | Get the EOAP associated with a deployed process.                                | Part 2     |
| Job status info                | `/jobs/{jobID}`                           | Retrieves the current status of a job.                                          | Part 1     |
| Job results                    | `/jobs/{jobID}/results`                   | Retrieves the results of a job.                                                 | Part 1     |
| Job list                       | `/jobs`                                   | Retrieves a list of submitted jobs.                                             | Part 1     |
| Job deletion                   | `/jobs/{jobID}` (DELETE)                  | Cancels and deletes a job.                                                      | Part 1     |


This resource permits the deployment of the an Application Package and provide two options for the `Content-Type`.

## Hands-on - Deploy the water_bodies Application Package

**Encoding Options**

The deployment can use two encodings that are based on the same CWL conformance class. Both methods utilize the same water_bodies.cwl file, but the way the file is provided differs:
- OGC Application Package Encoding (application/ogcapppkg+json): This method allows you to reference the CWL file by providing its location, rather than including the file's content in the request.
- CWL Encoding (application/cwl+yaml): This method requires the CWL file content to be included directly in the request body.

**Request Configuration**

When selecting a content type, the request body text area updates to contain a relevant payload template for that encoding.
Warning: If the payload is manually edited, switching to a different encoding may not refresh the text area. In this case, the Reset button can be used to restore the appropriate template.

**Server Response**

After executing the deployment request, the server responds with a process summary similar to the one obtained from the previous process listing endpoint.
The server’s response includes a Location header containing the URL to access the detailed process description.

**Next Steps** 

You can either:
- Return to the list of available processes to verify the newly deployed process.
- Proceed to the next step to review the process description in detail

## Setup

Lists available processes in the `acme` namespace.

> **_NOTE:_**: if the `acme` namespace does not exist, ZOO Project will create it.

In [1]:
import requests
import json
import yaml
import time
import os
import sys
from loguru import logger
from pystac.item_collection import ItemCollection
from pprint import pprint
namespace = "acme"

ogc_api_endpoint = f"http://zoo-project-dru-service/{namespace}/ogc-api"

r = requests.get(f"{ogc_api_endpoint}/processes")

r.status_code

200


If the application package was deployed previously, delete it.

In [2]:
def undeploy(process_id):

    r = requests.delete(f"{ogc_api_endpoint}/processes/{process_id}")

    return r

app_package_entry_point = "water-bodies"

r = undeploy(app_package_entry_point)

r.status_code

404

## OGC Application Package Encoding (application/ogcapppkg+json)

OGC Application Package Encoding (application/ogcapppkg+json): This method allows you to reference the CWL file by providing its location, rather than including the file's content in the request.

In [ ]:
app_package_entry_point = "water-bodies"
def app_package_deployment(app_package_entry_point, app_package_url):  
  headers = {"accept": "application/json", 
          "Content-Type": "application/ogcapppkg+json"}
  response = requests.post(
      
      f"{ogc_api_endpoint}/processes/?w={app_package_entry_point}",
      headers=headers,
      json = {
        "executionUnit": {
          "href": app_package_url,
          "type": "application/cwl"
        }
      }
  )
  return response

def check_app_package_deployment(app_package_entry_point):
    r = requests.get(f"{ogc_api_endpoint}/processes/")
   
    if r.status_code == 200:
        # Parse the JSON response
        all_processes = r.json()
        for process in all_processes.get("processes", []):
          if process.get("id") in [app_package_entry_point]:
              logger.info(f"Application package {app_package_entry_point} is already deployed.")
              return True
          else:
              logger.warning(f"{app_package_entry_point} still not deployed.")
              return False
    else:
        logger.error(f"Failed to retrieve processes. Status code: {r.status_code}")
        sys.exit(1)

def get_latest_application_package_version( repo_name):

    url = f"https://api.github.com/repos/eoap/{repo_name}/releases/latest"
    response = requests.get(url)
    response.raise_for_status()  # raise error if request failed

    latest_tag = response.json().get("tag_name")
    return latest_tag

In [ ]:
app_package_entry_point = "water-bodies"
is_package_deployed = check_app_package_deployment(app_package_entry_point)
repo_name = "mastering-app-package"
latest_application_package_version = get_latest_application_package_version(repo_name)
logger.info(f"Latest version is:  {latest_application_package_version}")
app_package_url = f"https://github.com/eoap/mastering-app-package/releases/download/{latest_application_package_version}/app-water-bodies-cloud-native.{latest_application_package_version}.cwl"
if not is_package_deployed:
    
    response= app_package_deployment(app_package_entry_point, app_package_url)
    pprint(response.json())

2025-08-20 13:40:19.690 | WARNING  | __main__:check_app_package_deployment:29 - water-bodies still not deployed.
2025-08-20 13:40:19.985 | INFO     | __main__:<module>:6 - Latest version is:  1.1.1


{'description': 'Water bodies detection based on NDWI and otsu threshold '
                'applied to Sentinel-2 COG STAC items',
 'id': 'water-bodies',
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'href': 'http://localhost:8080/acme/ogc-api/processes/water-bodies/execution',
            'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
            'title': 'Execute End Point',
            'type': 'application/json'}],
 'metadata': [{'role': 'https://schema.org/softwareVersion', 'value': '1.1.1'},
              {'role': 'https://schema.org/author',
               'value': {'@context': 'https://schema.org',
                         '@type': 'Person',
                         's.affiliation': 'ACME',
                         's.email': 'jane.doe@acme.earth',
                         's.name': 'Jane Doe'}}],
 'mutable': True,
 'outputTransmission': ['value', 'reference'],
 'title': 'Water bodies detection based on NDWI and otsu threshold',
 'version': '1.1.1'}


In [5]:
dict(response.headers)

{'Date': 'Wed, 20 Aug 2025 13:40:19 GMT',
 'Server': 'Apache/2.4.41 (Ubuntu)',
 'X-Powered-By': 'ZOO-Project-DRU',
 'X-Also-Powered-By': 'jwt.securityIn',
 'X-Also-Also-Powered-By': 'dru.securityIn',
 'Location': 'http://localhost:8080/acme/ogc-api/processes/water-bodies',
 'Keep-Alive': 'timeout=5, max=100',
 'Connection': 'Keep-Alive',
 'Transfer-Encoding': 'chunked',
 'Content-Type': 'application/json;charset=UTF-8'}

## CWL Encoding (application/cwl+yaml)

This method requires the CWL file content to be included directly in the request body.

If the application package was deployed previously, delete it.

In [6]:
app_package_entry_point = "water-bodies"

r = undeploy(app_package_entry_point)
r.status_code

204

Download the application package from https://github.com/eoap/mastering-app-package/releases/download/1.0.0/app-water-bodies-cloud-native.1.0.0.cwl

In [7]:
r = requests.get(app_package_url)

app_package_content  = yaml.safe_load(r.content)

app_package_content

{'cwlVersion': 'v1.0',
 '$namespaces': {'s': 'https://schema.org/'},
 's:softwareVersion': '1.1.1',
 'schemas': ['http://schema.org/version/9.0/schemaorg-current-http.rdf'],
 '$graph': [{'class': 'Workflow',
   'id': 'water-bodies',
   'label': 'Water bodies detection based on NDWI and otsu threshold',
   'doc': 'Water bodies detection based on NDWI and otsu threshold applied to Sentinel-2 COG STAC items',
   'requirements': [{'class': 'ScatterFeatureRequirement'},
    {'class': 'SubworkflowFeatureRequirement'}],
   'inputs': {'aoi': {'label': 'area of interest',
     'doc': 'area of interest as a bounding box',
     'type': 'string'},
    'epsg': {'label': 'EPSG code',
     'doc': 'EPSG code',
     'type': 'string',
     'default': 'EPSG:4326'},
    'stac_items': {'label': 'Sentinel-2 STAC items',
     'doc': 'list of Sentinel-2 COG STAC items',
     'type': 'string[]'},
    'bands': {'label': 'bands used for the NDWI',
     'doc': 'bands used for the NDWI',
     'type': 'string[]',
 

In [8]:
def app_package_deployment_cwl_encoding(app_package_entry_point, app_package_content):  
  
    headers = {"accept": "application/json", 
            "Content-Type": "application/cwl+yaml"}

    response = requests.post(
        f"{ogc_api_endpoint}/processes?w={app_package_entry_point}",
        headers=headers,
        json=app_package_content
    )
    print(response.status_code)
    return response

In [9]:
app_package_entry_point = "water-bodies"
is_package_deployed = check_app_package_deployment(app_package_entry_point)
if not is_package_deployed:
    response = app_package_deployment_cwl_encoding(app_package_entry_point, app_package_content)
    pprint(response.json())

2025-08-20 13:40:49.695 | WARNING  | __main__:check_app_package_deployment:29 - water-bodies still not deployed.


201
{'description': 'Water bodies detection based on NDWI and otsu threshold '
                'applied to Sentinel-2 COG STAC items',
 'id': 'water-bodies',
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'href': 'http://localhost:8080/acme/ogc-api/processes/water-bodies/execution',
            'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
            'title': 'Execute End Point',
            'type': 'application/json'}],
 'metadata': [{'role': 'https://schema.org/softwareVersion', 'value': '1.1.1'},
              {'role': 'https://schema.org/author',
               'value': {'@context': 'https://schema.org',
                         '@type': 'Person',
                         's.affiliation': 'ACME',
                         's.email': 'jane.doe@acme.earth',
                         's.name': 'Jane Doe'}}],
 'mutable': True,
 'outputTransmission': ['value', 'reference'],
 'title': 'Water bodies detection based on NDWI and otsu threshold',
 'version': '1.1.1

In [10]:
dict(response.headers)

{'Date': 'Wed, 20 Aug 2025 13:40:49 GMT',
 'Server': 'Apache/2.4.41 (Ubuntu)',
 'X-Powered-By': 'ZOO-Project-DRU',
 'X-Also-Powered-By': 'jwt.securityIn',
 'X-Also-Also-Powered-By': 'dru.securityIn',
 'Location': 'http://localhost:8080/acme/ogc-api/processes/water-bodies',
 'Keep-Alive': 'timeout=5, max=100',
 'Connection': 'Keep-Alive',
 'Transfer-Encoding': 'chunked',
 'Content-Type': 'application/json;charset=UTF-8'}

**Next Steps** 

You can either:
- Return to the list of available processes to verify the newly deployed process.
- Proceed to the next step to review the process description in detail